<center>
<img src="../logo.png" height="900">
</center>


#  Максимально правдоподобный скринкаст

In [ ]:
import numpy as np
import pandas as pd

from scipy import stats
from scipy.optimize import minimize

import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')  # more style :)
%matplotlib inline

# Что-то тут написал

# 1. Фрекен Бок

<center>
<img src="http://semyarf.com/UPLOAD/2016/01/21/frekenbok-216_700_0.jpg" height="200" width="350">
</center>

[Как известно](https://www.livelib.ru/quote/305456-malysh-i-karlson-kotoryj-zhivet-na-kryshe-astrid-lindgren), Фрекен Бок пьёт коньяк по утрам. У нас даже есть дневные данные (в граммах):

In [ ]:
x = [3.2, 7.9, 5.4, 4.9, 6.2, 4.3]

__а)__ Предполагая, что $x_i$ независимы и нормальны $N(\mu, \sigma^2)$ выпишем функцию правдоподобия.


$$
f(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \cdot e^{-\frac{(x - \mu)^2}{2 \sigma^2}}
$$




$$
L(\mu, \sigma^2 \mid x_1, \ldots, x_n) =  \frac{1}{(2 \pi \sigma^2)^{\frac{n}{2}}} \cdot e^{- \sum_{i=1}^n \frac{(x_i - \mu)^2}{2 \sigma^2}}
$$

$$
\ln L(\mu, \sigma^2 \mid x_1, \ldots, x_n) \propto  -0.5 \cdot n \cdot \ln  \sigma^2 - \sum_{i=1}^n \frac{(x_i - \mu)^2}{2 \sigma^2} \to \max_{\mu, \sigma^2}
$$

$\propto$ - равенство с точностью до константы

Чтобы гарантировать положительность параметра $\sigma^2$ функция у нас будет зависеть от вектора параметров $\theta$, причем $\theta_1 = \mu$, а $\theta_2 = \ln \sigma^2$.

In [ ]:
def lnL(theta, x):
    mu = theta[0]
    s2 = np.exp(theta[1])

    x =np.array(x)
    n = x.size

    l = -0.5*n*np.log(s2) - 0.5/s2*np.sum((x-mu)**2)
    return -l

lnL([4, 0.2], x)

10.240554617493236

__б)__ Оценим неизвестные параметры.

In [ ]:
theta_init = [0, 0]
res = minimize(lnL, theta_init, args=x)
res

      fun: 5.353606615954044
 hess_inv: array([[0.35268761, 0.01306079],
       [0.01306079, 0.31987723]])
      jac: array([-1.90734863e-06,  2.02655792e-06])
  message: 'Optimization terminated successfully.'
     nfev: 68
      nit: 14
     njev: 17
   status: 0
  success: True
        x: array([5.31666596, 0.78453622])

In [ ]:
mu, s2 = res.x
s2 = np.sqrt(np.exp(s2))
mu, s2

(5.316665963208802, 1.4803345526634897)

__в)__ Получим тот же результат используя встроенный метод `stats.norm.fit`:

In [ ]:
mu, s = stats.norm.fit(x)
mu, s

(5.316666666666666, 1.4803340463857775)

__г)__ Построим $95\%$ доверительный интервал для $\mu$

In [ ]:
res.hess_inv

array([[0.35268761, 0.01306079],
       [0.01306079, 0.31987723]])

In [ ]:
np.var(x)/len(x)

0.3652314814814815

In [ ]:
alpha = 0.05
z = stats.norm().ppf(1 - alpha/2)

left = res.x[0] - z*np.sqrt(res.hess_inv[0,0])
right = res.x[0] + z*np.sqrt(res.hess_inv[0,0])

print("Доверительный интервал [{:.4}; {:.4}] ширины {:.4}".format(left, right, right - left))

Доверительный интервал [4.153; 6.481] ширины 2.328


__д)__ Проверим гипотезу о том, что $\mu = 1$ с помощью теста отношения правдоподобий.

In [ ]:
def lnL_r(theta, x):
    mu = 1
    s2 = np.exp(theta)

    x =np.array(x)
    n = x.size

    l = -0.5*n*np.log(s2) - 0.5/s2*np.sum((x-mu)**2)
    return -l

lnL_r(0.2, x)

51.75020379854693

In [ ]:
theta_init = 0
res_r = minimize(lnL_r, theta_init, args=x)
res_r

      fun: 12.108462564158728
 hess_inv: array([[0.3327759]])
      jac: array([-2.38418579e-07])
  message: 'Optimization terminated successfully.'
     nfev: 30
      nit: 9
     njev: 10
   status: 0
  success: True
        x: array([3.03615412])

In [ ]:
lnL_r = res_r.fun
lnL_ur = res.fun

In [ ]:
2*(lnL_r - lnL_ur)

13.509711896409367

In [ ]:
stats.chi2(df=1).ppf(0.95)

3.841458820694124

Гипотеза отвергается!

__е)__ Проверим гипотезу о том, что $\mu = 1$, а $\sigma^2 = 2$ с помощью теста отношения правдоподобий.

In [ ]:
lnL_ur = res.fun
lnL_r = lnL([5,2.5], x)

In [ ]:
2*(lnL_r - lnL_ur)

5.42145549917052

In [ ]:
stats.chi2(df=2).ppf(0.95)

5.991464547107979

Гипотеза отвергается.

# 2. Призраки

<center>
<img src="https://pbs.twimg.com/media/DqWmCg9X4AAkl0n.jpg" height="200" width="350">
</center>

А ещё Фрекен-Бок иногда видит привидения! Данные по количеству привидений у нас тоже есть :)

In [ ]:
y =  [1, 2, 0, 0, 2, 0]

Предположим, что количество привидений имеет пуассоновское распределение с параметром $\lambda$.

__а)__ Оцените $\lambda$ с помощью ММП:

Вероятность, что случайная величина $X$ примет значение $k$:

$$
\mathbb{P}(X = k) = \frac{e^{-\lambda} \cdot \lambda^k}{k!}
$$

$$
L(\lambda \mid x_1, \ldots x_n) = \frac{e^{-\lambda} \cdot \lambda^{
x_1}}{x_1!} \cdot \ldots \cdot \frac{e^{-\lambda} \cdot \lambda^{x_n}}{x_n!}
$$

$$
\ln L(\lambda \mid x_1, \ldots x_n) = \cdot n + \ln \lambda \cdot \sum_{i=1}^n x_i - \sum_{i=1}^n x_i!  \propto - \lambda \cdot n + \ln \lambda \cdot \sum_{i=1}^n x_i \to \max_{\lambda}
$$

$\propto$ - равенство с точностью до константы


$$
\ln L(\lambda \mid x_1, \ldots x_n)  \propto  \sum_{i=1}^n [- \lambda + \ln \lambda \cdot x_i] \to \max_{\lambda}
$$

In [ ]:
def lnL(theta, y):
    rate = np.exp(theta)
    y = np.array(y)
    l = -rate + np.log(rate)*y
    return -1*np.sum(l)

lnL(4, y)

307.5889001988654

In [ ]:
theta_init = 0
res = minimize(lnL, theta_init, args=y)
res

      fun: 5.91160778397651
 hess_inv: array([[0.2006664]])
      jac: array([8.34465027e-06])
  message: 'Optimization terminated successfully.'
     nfev: 15
      nit: 3
     njev: 5
   status: 0
  success: True
        x: array([-0.18231992])

Предположим, что в $i$-й день интенсивность пуассоновского распределения $\lambda_i$ связана с количеством выпитого коньяка формулой $\lambda_i = e^{a + b \cdot x_i}$. То есть, возможно, что Фрекен Бок видит призраков из-за коньяка.

__б)__ Оценим параметры $a$ и $b$, выписав функцию правдоподобия.

$$
\ln L(\lambda \mid x_1, \ldots x_n)  \propto  \sum_{i=1}^n [- \lambda_i + \ln \lambda_i \cdot x_i] \to \max_{a, b}
$$

$$
\lambda_i = \exp(a + b y_i)
$$

In [ ]:
fbock = pd.DataFrame({
    'x': [3.2, 7.9, 5.4, 4.9, 6.2, 4.3],
    'y': [1, 2, 0, 0, 2, 0]
})

In [ ]:
def lnL(theta, fbock):
    x = np.array(fbock['x'])
    y = np.array(fbock['y'])

    a = theta[0]
    b = theta[1]

    rate = np.exp(a + b*x)
    l = -rate + np.log(rate)*y
    return -1*np.sum(l)

lnL([1,2], fbock)

20635012.00334143

In [ ]:
theta_init = [0,0]
res = minimize(lnL, theta_init, args=fbock)
res

      fun: 4.901374802427494
 hess_inv: array([[ 3.65178738, -0.54899458],
       [-0.54899458,  0.08731151]])
      jac: array([5.96046448e-08, 5.96046448e-08])
  message: 'Optimization terminated successfully.'
     nfev: 52
      nit: 9
     njev: 13
   status: 0
  success: True
        x: array([-2.59878438,  0.4169601 ])

__в)__ Построим прогнозы для парметра лямбда при разном объёме выпитого коньяка.

In [ ]:
x_test = np.array([1, 2])
lam_hat = np.exp(res.x[0] + res.x[1]*x_test)
lam_hat

array([0.1128355 , 0.17121004])

In [ ]:
# ФБ увидит 1 призрака
np.exp(-lam_hat)*lam_hat

array([0.10079568, 0.14426921])

По аналогии можем вычислить все интересующие нас вероятности.

__в)__  Данная модель является довольно популярной и реализована в `statsmodels`. Она называется Пуассоновской регрессией. Оценим её.

In [ ]:
!pip install statsmodels

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
import statsmodels.formula.api as smf

In [ ]:
model = smf.poisson(data=fbock, formula="y ~ 1 + x")
model.fit().summary()

Optimization terminated successfully.
         Current function value: 1.047945
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:                      y   No. Observations:                    6
Model:                        Poisson   Df Residuals:                        4
Method:                           MLE   Df Model:                            1
Date:                Wed, 12 Aug 2020   Pseudo R-squ.:                  0.1384
Time:                        18:17:08   Log-Likelihood:                -6.2877
converged:                       True   LL-Null:                       -7.2979
                                        LLR p-value:                    0.1552
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.5988      1.918     -1.355      0.175      -6.358       1.161
x              0.4170      0.297      1.404      0.160      -0.165       0.999
==============================================================================
"""

__г)__ C помощью полученного протокола, проверим гипотезу о взаимосвязи призраков и выпитого коньяка.

- Точечная оценка коэффицента перед коньяком равна $0.4170$. ММП даёт нам асимптотически нормальные оценки, то есть мы можем для проверки гипотез для коэффициента использовать $z-$тест.
- В колонке $z$ посчитана статистика для гипотезы о равенстве коэффициента нулю (гипотеза о значимости коэффициента). Она оказалась равна $1.404$. Также для неё найдено p_value, оно оказалось равно $0.160$. То есть гипотеза о равенстве коэффициента нулю не отвергается.

> Если все наши предпосылки выполнены и данные действительно имеют распределение Пуассона, коньяк никак не влияет на число увиденных приведений.


# 3. Гифка с накоплением

На гифке нарисовано как пуасоновская функция правдоподобия постепенно накапливается из отдельных наблюдений и становится более выраженной:

![ ](./animation_likelihood.gif)
